In [2]:
import os
from collections import Counter
import random
import yaml

base = '/media/michael/Seagate/engage/alison_data/golden_set/'
spec_pkl_dir = base + 'extracted/specs/'

In [5]:
files = [xx for xx in os.listdir(spec_pkl_dir) if xx.endswith('.pkl')]
file_sites = [xx.split('_')[0] for xx in files]
print len(files)
print len(set(file_sites))

40
19


In [6]:
site_counts = Counter(file_sites)
print site_counts

Counter({'E29RR-013378': 5, 'HA53AA-13548': 4, 'HA86RB-13527': 3, 'W112NN-13548': 3, 'TW76ER-13534': 3, 'CR05EF-13527': 2, 'DA5-011881': 2, 'BR28LB-11881': 2, 'CR8-13548': 2, 'BR67US-13534': 2, 'CM167NP-3527': 2, 'IG62XL-13534': 2, 'BR4-013378': 2, 'E105JP-13548': 1, 'SE3-13548': 1, 'SW154LA-3527': 1, 'BR20EG-13548': 1, 'WC2H8LG-3527': 1, 'SE23-13527': 1})


In [17]:
num_folds = 3

for seed in range(1000):
    print "Seed is ", seed
    sites = sorted(list(set(file_sites)))
    random.seed(seed)
    random.shuffle(sites)

    fold_size = len(sites) / num_folds
    file_fold_size = len(files) / num_folds

    # manually getting the 3 folds
    folds = []
    folds.append(sites[:fold_size])
    folds.append(sites[fold_size:2*fold_size])
    folds.append(sites[2*fold_size:])

    wav_folds = []
    
    passed = True

    for fold in folds:
        wav_fold_list = [xx.split('-sceneRect.csv')[0] 
                         for xx in files 
                         if xx.split('_')[0] in fold]
        wav_folds.append(wav_fold_list)

        num_files = sum([site_counts[xx] for xx in fold])
        print num_files
        if num_files < 13:
            passed = False
    if passed: break

Seed is  0
14
11
15
Seed is  1
15
13
12
Seed is  2
15
11
14
Seed is  3
11
14
15
Seed is  4
13
13
14


In [19]:
# saving the folds to disk
savedir = base + 'splits/'
for idx, (fold, wav_fold) in enumerate(zip(folds, wav_folds)):
    savepath = savedir + 'fold_sites_%d.yaml' % idx
    yaml.dump(fold, open(savepath, 'w'), default_flow_style=False)
    
    savepath = savedir + 'folds_%d.yaml' % idx
    yaml.dump(wav_fold, open(savepath, 'w'), default_flow_style=False)

In [21]:
savepath = savedir + 'fold_sites.yaml'
yaml.dump(folds, open(savepath, 'w'), default_flow_style=False)

savepath = savedir + 'folds.yaml'
yaml.dump(wav_folds, open(savepath, 'w'), default_flow_style=False)